# Lab 3 - n Puzzle (alternative solutions)

In these files there are 2 working solutions for the n puzzle problem.

They are not optimal in term of running time, so I've chosen to kept them separeted from the ones in the main file (nPuzzle.ipynb), as a bonus, just to show what I had tried to implement 

In [1]:
import numpy as np
import random
from tqdm.auto import tqdm
import time
import itertools
from operator import itemgetter

In [7]:
class nPuzzle():

    def __init__(self,d,n_shuffle=500):
        self.d = d
        val = np.arange(1,self.d**2)
        self.n = n_shuffle
        self.solution = np.append(val,0).reshape(self.d,self.d)
        self.board = self.solution.copy()
        self.puzzle_shuffle(self.n)

    def __str__(self):
        return f'{self.board}'
        
    def is_solved(self):
        if len(self.board)>0:
            return (self.board==self.solution).all()
        else:
            return False
        
    def find_void(self):
        pos = np.where(self.board==0)
        return (pos[0].item(), pos[1].item())
    
    def find_moves(self):
        r,c = self.find_void()
        
        if r in range(1,self.d-1) and c in range(1,self.d-1): # void cell not in contour
            return [(r,c-1),(r-1,c),(r+1,c),(r,c+1)]
        
        elif r in range(1,self.d-1) and c==0: # first column, middle rows
            return [(r-1,c),(r+1,c),(r,c+1)]
        
        elif r in range(1,self.d-1) and c==self.d-1: # last column, middle rows
            return [(r,c-1),(r-1,c),(r+1,c)]
        
        elif r==0 and c in range(1,self.d-1): # first row, middle columns
            return [(r,c-1),(r+1,c),(r,c+1)]
        
        elif r==self.d-1 and c in range(1,self.d-1): # last row, middle columns
            return [(r,c-1),(r-1,c),(r,c+1)]
        
        elif r==0 and c==0: # upper left corner
            return [(r+1,c),(r,c+1)]
        
        elif r==0 and c==self.d-1: # upper right corner
            return [(r+1,c),(r,c-1)]
        
        elif r==self.d-1 and c==0: # bottom left corner
            return [(r-1,c),(r,c+1)]
        
        elif r==self.d-1 and c==self.d-1: #bottom right corner
            return [(r,c-1),(r-1,c)]
        
        else:
            print("something went wrong!")
            return []
            
    def random_move(self):
        mat = self.board.copy()
        zero_pos = self.find_void()
        eval_moves = self.find_moves()
        next_pos = random.choice(eval_moves)
        self.board[zero_pos] = mat[next_pos] 
        self.board[next_pos] = 0
        return self.board
    
    def puzzle_shuffle(self,n):
        for _ in range(n):
            self.random_move()
        return self.board

In [8]:
# utility functions 

def is_solved(mat,solution):
    if len(mat)>0:
        return (mat==solution).all()
    else:
        return False
    
def find_void(mat):
    pos = np.where(mat==0)
    return (pos[0].item(), pos[1].item())

def find_moves(mat):
    r,c = find_void(mat)
    dim = len(mat)-1
    
    if r in range(1,dim) and c in range(1,dim): # void cell not in contour
        return [(r,c-1),(r-1,c),(r+1,c),(r,c+1)]
    
    elif r in range(1,dim) and c==0: # first column, middle rows
        return [(r-1,c),(r+1,c),(r,c+1)]
    
    elif r in range(1,dim) and c==dim: # last column, middle rows
        return [(r,c-1),(r-1,c),(r+1,c)]
    
    elif r==0 and c in range(1,dim): # first row, middle columns
        return [(r,c-1),(r+1,c),(r,c+1)]
    
    elif r==dim and c in range(1,dim): # last row, middle columns
        return [(r,c-1),(r-1,c),(r,c+1)]
    
    elif r==0 and c==0: # upper left corner
        return [(r+1,c),(r,c+1)]
    
    elif r==0 and c==dim: # upper right corner
        return [(r+1,c),(r,c-1)]
    
    elif r==dim and c==0: # bottom left corner
        return [(r-1,c),(r,c+1)]
    
    elif r==dim and c==dim: #bottom right corner
        return [(r,c-1),(r-1,c)]
    
    else:
        print("something went wrong!")
        return []

# function used to verify is a puzzle configuration
# has already been visited   
def already_visited(mat,visited_nodes):
    if len(visited_nodes)==0:
        return False
    else:
        for state in visited_nodes:
            is_duplicate = (mat==state).all()
            if is_duplicate:
                break
        return is_duplicate
    
# function used to checkif a puzzle configuration
# is visited 2+ times in a path (like the solution)    
def any_duplicates(path):
    known_state = []
    doubles_count = 0
    for el in path:
        if not already_visited(el,known_state):
            if len(known_state)==0:
                known_state = el.copy()
            else:
                known_state = np.concatenate((el,known_state))
        else:
            doubles_count += 1
    return doubles_count, known_state

In [9]:
# functions used to (try to) estimate the goodness of a move

# sum of Manhattan Distances of ALL wrong pieces to their correct positions (zero included)
# (this is the function implemented in all solutions)
def manhattan_distance(mat,solution):
    dist = 0
    if not is_solved(mat,solution):
        d1_board = mat.reshape(1,-1).squeeze()
        d1_solution = solution.reshape(1,-1).squeeze()
        for i,j in zip(d1_board,d1_solution):
            if i != j:
                a,b = np.where(mat==j)  
                c,d = np.where(solution==j)
                man_dist = abs(a-c) + abs(b-d)
                dist += man_dist
        return dist.item()
    
    else:
        return dist
        
# this function outputs the distance from zero-cell to first wrong element.
# the main idea is that, when two moves share the same goodness, to tie-break
# we use this function, because, in order to move a wrong piece, we need to have
# zero-cell near it. (maybe this is useless, I don't know)
def zero_fitness(mat):
    d1_board = mat.reshape(1,-1).squeeze()
    target=1 # we satart looking for element 1
    
    for i in range(len(d1_board)):
        el = d1_board[i]
        if el == target:
            target += 1
        else:
            break
            
    a,b = np.where(mat==target) # position of the first wrong number  
    c,d = np.where(mat==0) # position of zero
    fitness = abs(a-c) + abs(b-d) 
    
    if fitness.size>0:
        return fitness.item()
    else:
        return 0

## A* Search

The algorithm performs A* search. Queue and priority are kept separetd:
- Queue is a list of leafs (Board:np.array(2d), pos:int);
- Priority is a list ([int]).

At each step node with lowest priority is expanded and removed from queue, untill solution or empty queue.
The algorithm works fine with d=3, but it requires a lot of time when d>=4.
I've never run it for longer than 30 minutes, but it should find the solution, since it works with lower puzzle dimensions.

In [ ]:
# a simple data structure used to represent nodes
# the attibutes are:
# - puzzle configuration
# - position in the tree (with respect to the origin node)

class leaf():
    def __init__(self,board,pos):
        self.board=board
        self.pos=pos

    def __str__(self):
        return f"({self.board},{self.pos})"

In [10]:
# check if a node is in queue
def in_queue(mat,queue):
    if len(queue)==0:
        return False
    else:
        for leafs in queue:
            is_duplicate = (mat.board==leafs.board).all()
            if is_duplicate:
                break
        return is_duplicate
    
# check if a puzzle configuratuion is in queue multiple times (even with different priorities)
def any_duplicates_queue(path):
    known_state = []
    doubles_count = 0
    for el in path:
        if not in_queue(el,known_state):
            known_state.append(el)    
        else:
            doubles_count += 1
    return doubles_count, known_state
    
# function used to update priority
def update_priority(mat,queue,priority_list,sol):
    for i,leafs in enumerate(queue):
        if (mat.board==leafs.board).all() and mat.pos < leafs.pos:
            priority_list[i] = manhattan_distance(mat.board,sol) + mat.pos
    return priority_list

# function to generate never-before-seen child nodes.
def find_next_nodes(mat, prev_nodes=[]):
    start_board = mat.board.copy()
    start_pos = mat.pos
    known_boards = prev_nodes.copy()
    if len(known_boards)==0:
        known_boards = mat.board.reshape(1,len(start_board),len(start_board)).copy()
    
    zero_pos = find_void(start_board)
    candidate_moves = find_moves(start_board)
    eval_boards = []
    for move in candidate_moves:
        next_board = start_board.copy()
        next_board[zero_pos] = start_board[move] 
        next_board[move] = 0   
        
        if not already_visited(mat=next_board,visited_nodes=known_boards):
            eval_boards.append(leaf(next_board.copy(),start_pos+1))
            known_boards = np.concatenate((known_boards,next_board.reshape(1,len(start_board),len(start_board))))

    return eval_boards

In [11]:

def aStar(d,heuristic=manhattan_distance):
    puzzle = nPuzzle(d, n_shuffle=d**(2*(d-1)))
    print(puzzle)
    p_queue = [leaf(puzzle.board.copy(),0)]
    priority = [heuristic(p_queue[0].board,puzzle.solution)]
    eval_moves = []
    sol_path = []

    solved = is_solved(p_queue[0].board,puzzle.solution)
    if solved:
        print("Puzzle already solved!\nshuffle it to start playing.")
        sol_path.append(p_queue)
        
    start = time.time()
    while not solved:
        idx = np.argmin(priority)
        node = p_queue[idx] # board(array2d), pos(int)
        
        # add current node to visited nodes, if not already visited
        if not already_visited(mat=node.board,visited_nodes=eval_moves):
            if len(eval_moves)==0:
                eval_moves = node.board.reshape(1,puzzle.d,puzzle.d).copy()
            else:
                eval_moves = np.concatenate((eval_moves,node.board.reshape(1,puzzle.d,puzzle.d)))
                
        # add current node to solution path, if not already in
        #if not in_queue(node,sol_path):
        sol_path.append(node)
        
        # generating never visited child nodes
        next_nodes = find_next_nodes(mat=node, prev_nodes=eval_moves) #[board(array2d),pos(int)]
        
        # remove expanded node from queue
        p_queue.pop(idx)
        priority.pop(idx)
        assert len(p_queue) == len(priority)
            
        # insert child nodes (if any) in queue and checking for solution
        if len(next_nodes)>0:
            for n in next_nodes:
                if is_solved(n.board,puzzle.solution):
                    sol_path.append(n)
                    end = time.time()
                    print(f'found a solution in {end-start:.3f} seconds')
                    solved = True
                    break
                    
                #check if new nodes are in queue
                if not in_queue(n,p_queue):
                    # inserting new nodes in queue
                    p_queue.append(n)
                    priority.append(heuristic(n.board,puzzle.solution) + n.pos)
                    assert len(p_queue) == len(priority)
                # if already in query, update priority if lower* than existing
                # *in algorithm, priority is the estimated number of moves to the solution,
                # so items with lower priority are evaluated first
                else:
                    priority = update_priority(n,p_queue,priority,puzzle.solution)
                    
                
        # if no child node found -> backtrack -> remove current node from solution      
        else:
            sol_path = sol_path[:-1]
            
        # if we found solution in expanded nodes   
          
            
        # if no new nodes in queue -> puzzle not solvable -> stop search
        if len(p_queue)==0:
            print("Puzzle not solvable")
            break
    
    print(sol_path[-1].board)
    return sol_path

In [14]:
solution_path = aStar(d=3)

[[4 1 2]
 [7 8 6]
 [5 0 3]]
found a solution in 0.039 seconds
[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Depth First

Expand node starting from initial configuration.

At each step, node with lower heuristic is expanded.

When no new nodes found, backtrack.

Algorithm stops when solution is found or MAX_STEPS reached.


In [15]:
def find_next_nodes(mat, prev_nodes=[]):
    start_board = mat.copy()
    known_boards = prev_nodes.copy()
    if len(known_boards)==0:
        known_boards = mat.copy()
    
    zero_pos = find_void(start_board[0])
    candidate_moves = find_moves(start_board[0])
    eval_boards = []
    for move in candidate_moves:
        next_board = start_board[0].copy()
        next_board[zero_pos] = start_board[0][move] 
        next_board[move] = 0
        next_board = next_board.reshape(start_board.shape)   
        
        if not already_visited(mat=next_board,visited_nodes=known_boards):
            if len(eval_boards)==0:
                eval_boards = next_board.copy()
            else:
                eval_boards = np.concatenate((eval_boards,next_board))  
            known_boards = np.concatenate((known_boards,next_board))

    
    return eval_boards

In [23]:
def depth_first(d,heuristic=manhattan_distance, MAX_STEPS=1_000):
    puzzle = nPuzzle(d, n_shuffle=d**(2*(d-1)))
    print(puzzle)
    p_queue = puzzle.board.reshape(1,puzzle.d,puzzle.d).copy()
    eval_moves = []
    sol_path = []

    start = time.time()
    for _ in tqdm(range(MAX_STEPS)):
        node = p_queue[0].reshape(1,puzzle.d,puzzle.d)
        
        # add node to solution path, if not already in
        if not already_visited(mat=node,visited_nodes=sol_path):
            if len(sol_path)==0:
                sol_path = node.copy()
            else:
                sol_path = np.concatenate((sol_path,node))
            
        # add current node to visited nodes, if not already visited
        if not already_visited(mat=node,visited_nodes=eval_moves):
            if len(eval_moves)==0:
                eval_moves = node.copy()
            else:
                eval_moves = np.concatenate((eval_moves,node))
            
        # generating never visited child nodes
        next_nodes = find_next_nodes(mat=node, prev_nodes=eval_moves)
        
        # remove expanded node from queue
        if len(p_queue)==1:
            p_queue = []  
        else:
            p_queue = p_queue[1:]
            
        # sort child nodes (if any) and checking for solution
        if len(next_nodes)>0:
            heuristic_scores = [heuristic(el,puzzle.solution) for el in next_nodes]
            ind = np.argsort(heuristic_scores)
            next_nodes = next_nodes[ind] 
            if next_nodes.shape==(puzzle.d,puzzle.d):
                next_nodes = next_nodes.reshape(1,puzzle.d,puzzle.d)
                
            # inserting new nodes in queue, with highest priority 
            if len(p_queue)==0:
                p_queue = next_nodes.copy()
            else:
                p_queue = np.concatenate((next_nodes,p_queue))
                
            # checking if node with highest priority is solution
            if is_solved(p_queue[0],puzzle.solution):
                sol_path = np.concatenate((sol_path,p_queue[0].reshape(1,puzzle.d,puzzle.d)))
                end = time.time()
                print(f'found a solution of {len(sol_path)} moves in {end-start:.3f} seconds')
                break
                
        # if no child node found -> backtrack -> remove current node from solution      
        else:
            sol_path = sol_path[:-1]
            
        # if no new nodes in queue -> puzzle not solvable -> stop search
        if len(p_queue)==0:
            print("Puzzle not solvable")
            break
       
    return sol_path


In [24]:
solution = depth_first(d=3)

[[4 0 1]
 [7 2 5]
 [6 8 3]]


  0%|          | 0/1000 [00:00<?, ?it/s]

found a solution of 330 moves in 0.726 seconds


Again, it works fine with d=3.
Whan d grows, it keeps running and running...